Step 1: Load packages

In [1]:
using JuMP
using Clp

Step 2: Defining a primal model

I'll discuss each step in comments

In [2]:
"""
Initialize the model and set the optimizer. In this case we are using Clp. 
To see all solvers JuMP supports visit here:

https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers

Note: Some of these are just calls to C code and may be licensed. Clp and Ipopt
are free.
"""

primal = Model(Clp.Optimizer)

"""
Initialize the variables with bounds. 

This is a macro, which is a Julia thing. Not important to understand 100% now, 
but it's good to know the words. 

This is a plural macro, it ends in s, which means it'll define multiple variables
each on a new line. 

We could also do:

@variable(primal,0<=X<=1000)
@variable(primal, 0<=Y<=1750)

but it's slightly more typing. Most Julia macros have this.
"""

@variables(primal,begin
    0<=X<=1000 #productoin of lager
    0<=Y<=1750 #production of ale
end)

"""
Set up the objective function. In GAMS, the objective has a corresponding 
variable, not the case here.

Instead of Max, we could have Min. But we're not minimizing...
"""

@objective(primal, Max, 120*X + 90*Y)

"""
Add the constrants. The second coordinate is the name of the contraint.

There is a constraints macro, I use this in the dual verson.
"""

@constraint(primal,malt, 4*X+2*Y <= 4800)

@constraint(primal, yeast, X+Y <=1750)


"""
Optmize the model. Notice the ! at the end of optimize. This is a Julia
convention telling you that the input is changing. 
"""

optimize!(primal)

"""
Optional, view the solution summary.
"""

solution_summary(primal)


Coin0506I Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
Clp0006I 0  Obj 0 Dual inf 210 (2)
Clp0006I 2  Obj 177000
Clp0000I Optimal - objective value 177000
Clp0032I Optimal objective 177000 - 2 iterations time 0.002


* Solver : Clp

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "0 - optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.77000e+05

* Work counters
  Solve time (sec)   : 0.00000e+00


In [3]:
"""
Just a quick and dirty way to view all the solutions. 

Notice the \$ inside the string, this pulls the actual value, so
\$ var is replaced by the actual variable
"""

for var in all_variables(primal)
    println("$var -> $(value(var))")
end

X -> 650.0
Y -> 1100.0


In [4]:
println("malt constraint value -> $(value(primal[:malt]))")
println("yeast constraint value -> $(value(primal[:yeast]))")

malt constraint value -> 4800.0
yeast constraint value -> 1750.0


# Dual

Julia supports _any_ unicode character as a variable name. 

To get the fancy characters do this:

\lambda

and press tab

Subscripts are 

\_number (tab)

However, I hate typing lambda, so I'll use L instead of lambda

In [5]:
dual = Model(Clp.Optimizer)

@variables(dual,begin
    λ₁ >= 0
    L2 >= 0
    L3 >= 0
    L4 >= 0
end)

@objective(dual,Min, 4800*λ₁ + 1750*L2 + 1000*L3 + 1500*L4)


"""
Here is the constraints macro. Each constraint is on a new line.
"""

@constraints(dual,begin
    dualX, 4*λ₁ + 1*L2 + L3 + 0*L4 >= 120
    dualY, 2*λ₁ + 1*L2 + 0*L3 + 1*L4 >= 90
end)

optimize!(dual)

solution_summary(dual)

Coin0506I Presolve 2 (0) rows, 4 (0) columns and 6 (0) elements
Clp0006I 0  Obj 0 Primal inf 75 (2)
Clp0006I 3  Obj 177000
Clp0000I Optimal - objective value 177000
Clp0032I Optimal objective 177000 - 3 iterations time 0.002


* Solver : Clp

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "0 - optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.77000e+05

* Work counters
  Solve time (sec)   : 0.00000e+00


In [6]:
for var in all_variables(dual)
    println("$var -> $(value(var))")
end

λ₁ -> 15.0
L2 -> 60.0
L3 -> 0.0
L4 -> 0.0


In [7]:
println("dualX -> $(value(dual[:dualX]))")
println("dualY -> $(value(dual[:dualY]))")

dualX -> 120.0
dualY -> 90.0
